In [1]:
#|default_exp doclinks

# doclinks
- Generating a documentation index from a module

In [2]:
#|export
from nbdev.read import *
from nbdev.maker import *
from nbdev.export import *
from nbdev.imports import *

from fastcore.script import *
from fastcore.imports import *
from fastcore.basics import *
from fastcore.imports import *
from fastcore.meta import delegates

import ast,contextlib
import pkg_resources,importlib

from pprint import pformat
from urllib.parse import urljoin
from importlib import import_module

if IN_NOTEBOOK:
    from IPython.display import Markdown,display
    from IPython.core import page
else: Markdown,display,page = None,None,None

In [3]:
#|hide
from fastcore.test import *
from pdb import set_trace
from importlib import reload

## Creating the module index

In [4]:
#|export
def _mod_fn2name(fn):
    "Convert filename `fn` to its module name"
    return '.'.join(str(Path(fn).with_suffix('')).split('/'))

class DocLinks:
    "Create a module symbol index from a python source file"
    def __init__(self, mod_fn, doc_func, dest_fn, mod_name=None):
        mod_fn,dest_fn = Path(mod_fn),Path(dest_fn)
        if mod_name is None: mod_name = _mod_fn2name(
            mod_fn.resolve().relative_to(dest_fn.parent.parent.resolve()))
        store_attr()
        if self.dest_fn.exists(): self.d = exec_local(self.dest_fn.read_text(), 'd')
        else: self.d = dict(syms={}, settings={}) 

The doc index has to be stored in a file. Usually we call it `_modidx.py`. For testing, we'll delete any existing file first.

In [5]:
dest_fn = Path('tmp/_modidx.py')
with contextlib.suppress(FileNotFoundError): dest_fn.unlink()

A link to docs is created by a `doc_func`. We'll use a dummy one for testing.

In [6]:
def _help(m, s=None): return f"help for {m}; {s}"

We're now ready to instantiate `DocLinks` for our test module.

In [7]:
mod_fn = Path('tmp/everything.py')
link = DocLinks(mod_fn, _help, dest_fn)
link.mod_name

'tmp.everything'

In [8]:
#|export
@patch
def write_nbdev_idx(self:DocLinks):
    "Create nbdev documentation index file`"
    res = pformat(self.d, width=160, indent=2, compact=True)
    self.dest_fn.write_text("# Autogenerated by nbdev\n\nd = " + res)

Initially the index file will contain empty `syms` and `settings`:

In [9]:
tmp_path = Path('tmp')
tmp_path.mkdir(exist_ok=True)
link.write_nbdev_idx()
assert "Autogenerated" in dest_fn.read_text()

print(Path('tmp/_modidx.py').read_text())

# Autogenerated by nbdev

d = {'settings': {}, 'syms': {}}


In [10]:
#|export
def _binop_leafs(bo, o):
    if isinstance(bo.left, ast.BinOp): left = _binop_leafs(bo.left, o)
    else: left = [f'{bo.left.id}.{o.name}']
    if isinstance(bo.right, ast.BinOp): right = _binop_leafs(bo.right, o)
    else: right = [f'{bo.right.id}.{o.name}']
    return concat(left + right)

In [11]:
#|export
def _all_or_exports(fn):
    code = Path(fn).read_text()
    trees = L(ast.parse(code).body)
    res = read_var(code, '__all__')
    return L(retr_exports(trees) if res is None else res),trees

def _get_patch(o):
    if not isinstance(o, (ast.FunctionDef,ast.AsyncFunctionDef)): return
    return first([d for d in o.decorator_list if decor_id(d).startswith('patch')])

def get_patch_name(o):
    d = _get_patch(o)
    if not d: return
    nm = decor_id(d)
    if nm=='patch': 
        a = o.args.args[0].annotation
        if isinstance(a, ast.BinOp): return _binop_leafs(a, o)
        else: pre = a.id        
    elif nm=='patch_to': pre = o.decorator_list[0].args[0].id
    else: return
    return f'{pre}.{o.name}'

In [12]:
s = """class _T: pass
@patch
def _f(self:_T): pass
@patch_to(_T)
def _g(self): pass"""

res = [get_patch_name(o) for o in ast.parse(s).body]
test_eq([None, '_T._f', '_T._g'], res)

In [13]:
#|hide

#When you do a patch with a union
s = """
class _T: pass
class _U: pass
class _V: pass

@patch
def _f(self:_T|_U|_V): pass
"""
res = L(concat([get_patch_name(o) for o in ast.parse(s).body])).filter()
test_eq(res, ['_T._f', '_U._f', '_V._f'])

In [14]:
#|export
def _exp_meths(tree):
    return L(f"{tree.name}.{o.name}" for o in tree.body
             if isinstance(o,(ast.FunctionDef,ast.AsyncFunctionDef)) and o.name[0]!='_')

@patch
def update_syms(self:DocLinks):
    exp,trees = _all_or_exports(self.mod_fn)
    exp_class = trees.filter(lambda o: isinstance(o, ast.ClassDef) and o.name in exp)
    exp += exp_class.map(_exp_meths).concat()
    exp += L(concat([get_patch_name(o) for o in trees])).filter()
    exp = exp.map(f"{self.mod_name}.{{}}")
    self.d['syms'][self.mod_name] = exp.map_dict(partial(self.doc_func, self.mod_name))

In [15]:
everything_fn = '../tests/01_everything.ipynb'
nb_export('../tests/00_some.thing.ipynb', 'tmp')
nb_export(everything_fn, 'tmp')

In [16]:
link.update_syms()
link.write_nbdev_idx()

In [17]:
#|eval: false
g = exec_new('import tmp._modidx')
d = g['tmp']._modidx.d
symn = 'tmp.everything.a_y'
mod_name = 'tmp.everything'
test_eq(d['syms'][mod_name][symn], _help(mod_name,symn))
test_eq(set(d['syms'][mod_name].keys()),
        set(L('m_y', 'n_y', 'q_y', 'a_y', 'b_y', 'd_y', 'e_y', 'o_y', 'p_y', 'd_y.di_n', 'd_y.d3i_n', 'd_y.d4i_n'
             ).map('tmp.everything.{}')))

In [18]:
#|export
@patch
def build_index(self:DocLinks):
    self.update_syms()
    self.d['settings'] = dict(**get_config().d)
    self.write_nbdev_idx()

In [19]:
#|eval: false
link.build_index()
del(sys.modules['tmp._modidx'])
g = exec_new('import tmp._modidx')
d = g['tmp']._modidx.d
test_eq(d['settings']['lib_name'], 'nbdev')

In [20]:
#|export
def _doc_link(url, mod, sym=None):
    res = urljoin(url, remove_prefix(mod, get_config()['lib_name']+"."))
    if sym: res += "#" + remove_prefix(sym, mod+".")
    return res

In [21]:
#|export
def build_modidx():
    "Create _modidx.py"
    dest = config_key('lib_path')
    if os.environ.get('IN_TEST',0): return
    _fn = dest/'_modidx.py'
    nbs_path = config_key('nbs_path')
    files = globtastic(nbs_path)
    with contextlib.suppress(FileNotFoundError): _fn.unlink()
    cfg = get_config()
    doc_func = partial(_doc_link, urljoin(cfg.doc_host,cfg.doc_baseurl))
    for file in dest.glob("**/*.py"):
        if file.name[0]!='_': DocLinks(file, doc_func, _fn).build_index()

In [22]:
#|export
@delegates(globtastic, but=['file_glob', 'skip_folder_re'])
def nbglob(path=None, skip_folder_re = '^[_.]', file_glob='*.ipynb', recursive=True, key='nbs_path', as_path=False, **kwargs):
    "Find all files in a directory matching an extension given a `config_key`."
    path = Path(path or config_key(key))
    if recursive is None: recursive=get_config().get('recursive', 'False').lower() == 'true'
    res = globtastic(path, file_glob=file_glob, skip_folder_re=skip_folder_re, **kwargs)
    return res.map(Path) if as_path else res

In [23]:
#|export
@call_parse
def nbdev_export(
    path:str=None, # path or filename
    recursive:bool=None, # search subfolders
    symlinks:bool=True, # follow symlinks?
    file_re:str=None, # Only include files matching regex
    folder_re:str=None, # Only enter folders matching regex
    skip_file_glob:str=None, # Skip files matching glob
    skip_file_re:str=None # Skip files matching regex
):
    "Export notebooks in `path` to python modules"
    if os.environ.get('IN_TEST',0): return
    files = nbglob(path=path, recursive=recursive, file_re=file_re, 
                   folder_re=folder_re, skip_file_glob=skip_file_glob, skip_file_re=skip_file_re, symlinks=symlinks)
    for f in files:
        nb_export(f)
    add_init(get_config().path('lib_path'))
    build_modidx()

In [24]:
#|hide
from fastcore.test import *

In [25]:
#|export
def _settings_libs():
    try: #settings.ini doesn't exist yet until you call nbdev_new
        cfg = get_config()
        return cfg.get('strip_libs', cfg.get('lib_name')).split()
    except FileNotFoundError: return 'nbdev'

In [26]:
#|export
class NbdevLookup:
    "Mapping from symbol names to URLs with docs"
    def __init__(self, strip_libs=None, incl_libs=None, skip_mods=None):
        if strip_libs is None: strip_libs = _settings_libs()
        skip_mods = setify(skip_mods)
        strip_libs = L(strip_libs)
        if incl_libs is not None: incl_libs = (L(incl_libs)+strip_libs).unique()
        # Dict from lib name to _nbdev module for incl_libs (defaults to all)
        self.entries = {o.name: o.load() for o in pkg_resources.iter_entry_points(group='nbdev')
                       if incl_libs is None or o.dist.key in incl_libs}
        py_syms = merge(*L(o['syms'].values() for o in self.entries.values()).concat())
        for m in strip_libs:
            if m in self.entries:
                _d = self.entries[m]
                stripped = {remove_prefix(k,f"{mod}."):v
                            for mod,dets in _d['syms'].items() if mod not in skip_mods
                            for k,v in dets.items()}
                py_syms = merge(stripped, py_syms)
        self.syms = py_syms

    def __getitem__(self, s): return self.syms.get(s, None)

Symbol names are taken from libraries registered using the 'nbdev' entry point. By default, all libraries with this entry point are searched, but full symbol names (including module prefix) are required.

In [29]:
c = NbdevLookup()
assert c['nbdev.doclinks.DocLinks'].startswith('http')
assert c['numpy.array'].startswith('http')
assert c['DocLinks'].startswith('http')
assert not c['array']

Pass `strip_libs` to list libraries which should be available without requiring a module prefix.

In [30]:
c = NbdevLookup(strip_libs=['nbdev', 'nbdev_numpy'])
assert c['array'].startswith('http')
assert c['DocLinks'].startswith('http')

nbdev itself includes `nbdev_lookup`, an instantiated `NbdevLookup` with `strip_libs=nbdev`.

In [31]:
_nbdev_lookup = NbdevLookup()
assert _nbdev_lookup['DocLinks'].startswith('http')
assert _nbdev_lookup['numpy.array'].startswith('http')
assert not _nbdev_lookup['array']

## Backticks

In [32]:
#|export
@patch
def _link_sym(self:NbdevLookup, m):
    l = m.group(1)
    s = self[l]
    if s is None: return m.group(0)
    l = l.replace('\\', r'\\')
    return rf"[{l}]({s})"

_re_backticks = re.compile(r'`([^`\s]+)`')
@patch
def link_line(self:NbdevLookup, l): return _re_backticks.sub(self._link_sym, l)

@patch
def linkify(self:NbdevLookup, md):
    if md:
        in_fence=False
        lines = md.splitlines()
        for i,l in enumerate(lines):
            if l.startswith("```"): in_fence=not in_fence
            elif not l.startswith('    ') and not in_fence: lines[i] = self.link_line(l)
        return '\n'.join(lines)

In [33]:
md = """This is a link to `numpy.array` and to `get_config` but not a link to `foobar`.
And not a link to <code>dict2nb</code>.

    This is not a link to `get_config`

```
This isn't a link to `get_config` either
```"""

In [34]:
#|eval: false
c = NbdevLookup('nbdev')
Markdown(c.linkify(md))

This is a link to [numpy.array](https://numpy.org/doc/stable/reference/generated/numpy.array.html#numpy.array) and to [get_config](https://nbdev.fast.ai/read#get_config) but not a link to `foobar`.
And not a link to <code>dict2nb</code>.

    This is not a link to `get_config`

```
This isn't a link to `get_config` either
```

## Export -

In [35]:
#|eval: false
Path('../nbdev/export.py').unlink(missing_ok=True)
nbdev_export()

g = exec_new('import nbdev.export')
assert hasattr(g['nbdev'].export, 'nb_export')
from nbdev._modidx import d
assert d['syms']['nbdev.doclinks']['nbdev.doclinks.DocLinks'].startswith('http')